In [ ]:
%pip install python-dotenv requests pandas


## Introduction
Simple program to grab transactions from Lunchmoney.app. The file is expecting a `.env` to exist with the following variables:
```
LM_API_KEY="123456"
LM_HOSTNAME="https://dev.lunchmoney.app"
```

In [ ]:
from dotenv import load_dotenv
import os
import requests
import json
import csv

# Load the .env file
load_dotenv()

# Get the value of the 'LM_API' environmental variable
lm_api = os.getenv('LM_API_KEY')
lm_hostname = os.getenv('LM_HOSTNAME')

# Set the headers for the request
headers = {
    "Authorization": f"Bearer {lm_api}"
}

# Get the transactions from /v1/transactions. The format is JSON.
url = f"{lm_hostname}/v1/transactions"

response = requests.get(url, headers=headers)

"""  { "transactions": [
    {
      "id": 602,
      "date": "2020-01-01",
      "payee": "Starbucks",
      "amount": "4.5000",
      "currency": "cad",
      "to_base": 4.5,
      "notes": "Frappuccino",
      "category_id": null,
      "recurring_id": null,
      "asset_id": null,
      "plaid_account_id": null,
      "status": "cleared",
      "is_group": false,
      "group_id": null,
      "parent_id": null,
      "external_id": null,
      "original_name": "STARBUCKS NW 32804",
      "type": null,
      "subtype": null,
      "fees": null,
      "price": null,
      "quantity": null
    },
    {
      "id": 603,
      "date": "2020-01-02",
      "payee": "Walmart",
      "amount": "20.9100",
      "to_base": 20.91,
      "currency": "usd",
      "notes": null,
      "category_id": null,
      "recurring_id": null,
      "asset_id": 153,
      "plaid_account_id": null,
      "status": "uncleared",
      "is_group": false,
      "group_id": null,
      "parent_id": null,
      "external_id": "jf2r3t98o943",
      "original_name": "Walmart Superstore ON 39208",
      "type": null,
      "subtype": null,
      "fees": null,
      "price": null,
      "quantity": null
    }
  ]
} """

# write all the transactions to a file called 'transactions.csv'
transactions = response.json()['transactions']

with open('transactions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['date', 'payee', 'amount'])
    for transaction in transactions:
        writer.writerow([transaction['date'], transaction['payee'], transaction['amount']])


In [ ]:
url = f"{lm_hostname}/v1/assets"

response = requests.get(url, headers=headers)
assets = response.json()

print(json.dumps(assets, indent=2))



In [ ]:
# Write a GET request to v1/budgets endpoint using the requests library and the lm_hostname and lm_api as the bearer token
# Pass the 'start_date' and 'end_date' query parameters as '2023-11-01' and '2023-12-31' respectively
url = f"{lm_hostname}/v1/budgets"

params = {
    "start_date": "2023-11-01",
    "end_date": "2023-12-31"
}

response = requests.get(url, headers=headers, params=params)
print(json.dumps(response.json(), indent=2))

## Summary based on time of month
Get a summary of transactions and output the difference between the spending this month, so far, against the same period last month. Make sure it is as a percentage of the days in that month. The output should be `Diff: -$52 and -3%`

In [ ]:
import pandas as pd

# using pandas, get the date of the start of the month and then the date of the previous month
today = pd.to_datetime('today')
start_of_month = today.replace(day=1)
end_of_month = start_of_month - pd.Timedelta(days=1)
start_of_previous_month = start_of_month - pd.offsets.MonthBegin(1)

percentage_of_current_month = today.day / end_of_month.day
equivalent_day_of_previous_month = start_of_previous_month + pd.Timedelta(days=(today.day / end_of_month.day) * start_of_previous_month.days_in_month)

print (today, equivalent_day_of_previous_month)

url = f"{lm_hostname}/v1/transactions"

params = {
    "start_date": start_of_month.strftime('%Y-%m-%d'),
    "end_date": today.strftime('%Y-%m-%d')
}

response = requests.get(url, headers=headers, params=params)

# sum the amount of all the transactions for the current month
transactions = response.json()['transactions']

this_month_total = 0
for transaction in transactions:
    this_month_total += float(transaction['amount'])

params = {
    "start_date": start_of_previous_month.strftime('%Y-%m-%d'),
    "end_date": equivalent_day_of_previous_month.strftime('%Y-%m-%d')
}

## TODO: Make just one API request and then filter based on dates locally
response = requests.get(url, headers=headers, params=params)

# sum the amount of all the transactions for the previous month
transactions = response.json()['transactions']
last_month_total = 0
for transaction in transactions:
    last_month_total += float(transaction['amount'])

print(this_month_total)
print(last_month_total)

print('Diff in total: ', this_month_total - last_month_total, 'Diff in percentage: ', (this_month_total - last_month_total) / last_month_total * 100, '%')




### Second task
Take a look at the budgets endpoint

In [ ]:
# GET https://dev.lunchmoney.app/v1/budgets

url = f"{lm_hostname}/v1/budgets"

params = {
    "start_date": start_of_month.strftime('%Y-%m-%d'),
    "end_date": today.strftime('%Y-%m-%d')
}

try:
    # Make the GET request to the API
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        budget_data = response.json()

        # Initialize a dictionary to store the category names and their sums
        category_sums = {}

        # Iterate over each category
        for category in budget_data:
            category_name = category['category_name']
            total_spending_to_base = 0

            # Check if the 'data' field exists and is a dictionary
            if 'data' in category and isinstance(category['data'], dict):
                # Iterate over each date in the category
                for date_data in category['data'].values():
                    # Check if the 'spending_to_base' key exists
                    if 'spending_to_base' in date_data:
                        total_spending_to_base += date_data['spending_to_base']

            # Add the category name and sum to the dictionary
            category_sums[category_name] = total_spending_to_base

        # Print out the category names and sums
        total_sums = 0
        for category_name, total_spending_to_base in category_sums.items():
            total_sums += total_spending_to_base
            if (total_spending_to_base > 0):
                print(f"Category Name: {category_name} - Total Spending to Base: {total_spending_to_base}")
        print (total_sums)

    else:
        # Handle errors
        response.raise_for_status()

except requests.exceptions.RequestException as e:
    # Handle exceptions
    print(e)



